In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os

os.chdir("/content/gdrive/My Drive/rsna/")
os.getcwd()

'/content/gdrive/My Drive/rsna'

In [ ]:
import matplotlib.pyplot as plt
import cv2
import glob, pylab, pandas as pd
import numpy as np
%matplotlib inline

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
df_detailed = pd.read_csv('./stage_2_detailed_class_info.csv')

In [ ]:
df_detailed.head()

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity


In [ ]:
df_detailed['patientId'] =   df_detailed['patientId'].astype(str) + '.png' 

In [ ]:
image_gen = ImageDataGenerator(rotation_range = 30,
                              width_shift_range= 0.1, 
                              height_shift_range= 0.1,
                              rescale= 1/255,
                              shear_range=0.2,
                              zoom_range= 0.2,
                              horizontal_flip= True,
                              fill_mode='nearest',
                               validation_split =0.2
                              )
                              

In [ ]:
train_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(256,256),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=64,
                              shuffle=True,
                              subset = 'training'
                             )

Found 24015 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 209 invalid image filename(s) in x_col="patientId". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
test_image_gen = image_gen.flow_from_dataframe(dataframe = df_detailed,
                             directory= './train',
                             x_col = 'patientId',
                             y_col = 'class',
                              target_size=(256,256),
                              color_mode='rgb',
                              classes= None,
                              class_mode='categorical',
                              batch_size=64,
                              shuffle=True,
                              subset = 'validation'
                             )

Found 6003 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 209 invalid image filename(s) in x_col="patientId". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [ ]:
from tensorflow import keras
from keras.applications import DenseNet121
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Conv3D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.models import Model

In [ ]:
orig_net = DenseNet121(include_top=False, weights='imagenet', input_shape=(256,256,3)) 


In [ ]:
filters = GlobalAveragePooling2D()(orig_net.output) 

In [ ]:
classifiers = Dense(3, activation='softmax', bias_initializer='ones')(filters)  

In [ ]:
model = Model(inputs=orig_net.inputs, outputs=classifiers)  


In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("dense121_1.h5", monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto',
                             period=1)

early = EarlyStopping(monitor='val_accuracy', 
                      min_delta=0, 
                      patience=20, 
                      verbose=1, 
                      mode='auto')

results = model.fit_generator(train_image_gen, epochs =150, 
                              steps_per_epoch=200,
                             validation_data=test_image_gen,  
                              callbacks=[checkpoint,early])

Epoch 1/150
 66/200 [========>.....................] - ETA: 52:36 - loss: 0.8801 - accuracy: 0.5866

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 104 could not be retrieved. It could be because a worker has died.
  UserWarning)


197/200 [============================>.] - ETA: 1:08 - loss: 0.7749 - accuracy: 0.6348

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 274 could not be retrieved. It could be because a worker has died.
  UserWarning)


200/200 [==============================] - 5945s 30s/step - loss: 0.7742 - accuracy: 0.6355 - val_loss: 1.6263 - val_accuracy: 0.4886

Epoch 00001: val_accuracy improved from -inf to 0.48859, saving model to dense121_1.h5
Epoch 2/150
 82/200 [===========>..................] - ETA: 35:02 - loss: 0.6856 - accuracy: 0.6923

...

137/200 [===================>..........] - ETA: 19:54 - loss: 0.6904 - accuracy: 0.6842

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 201 could not be retrieved. It could be because a worker has died.
  UserWarning)


152/200 [=====================>........] - ETA: 15:23 - loss: 0.6936 - accuracy: 0.6825

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 71 could not be retrieved. It could be because a worker has died.
  UserWarning)


200/200 [==============================] - 4115s 21s/step - loss: 0.6824 - accuracy: 0.6880 - val_loss: 1.0417 - val_accuracy: 0.5616

Epoch 00002: val_accuracy improved from 0.48859 to 0.56155, saving model to dense121_1.h5
Epoch 3/150
200/200 [==============================] - 3670s 18s/step - loss: 0.6563 - accuracy: 0.6994 - val_loss: 0.9740 - val_accuracy: 0.4216

Epoch 00003: val_accuracy did not improve from 0.56155
Epoch 4/150
200/200 [==============================] - 3691s 18s/step - loss: 0.6659 - accuracy: 0.6887 - val_loss: 0.6538 - val_accuracy: 0.6742

Epoch 00004: val_accuracy improved from 0.56155 to 0.67416, saving model to dense121_1.h5
Epoch 5/150
200/200 [==============================] - 3815s 19s/step - loss: 0.6488 - accuracy: 0.7059 - val_loss: 0.8434 - val_accuracy: 0.5830

Epoch 00005: val_accuracy did not improve from 0.67416
Epoch 6/150
200/200 [==============================] - 3563s 18s/step - loss: 0.6408 - accuracy: 0.7057 - val_loss: 0.9065 - val_accur

In [ ]:
plt.plot(results.history["loss"], label='Train Loss')
plt.plot(results.history["val_loss"], label='Validation loss')
plt.legend();

In [ ]:
plt.plot(results.history["accuracy"], label='Train accuracy')
plt.plot(results.history["val_accuracy"], label='Validation accuracy')
plt.legend();